In [1]:
# !pip install --upgrade google-cloud-speech

In [3]:
# !pip install rasa_nlu

In [8]:
# !pip install wit

In [70]:
!pip install --upgrade google.cloud

  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl


In [60]:
!pip install --upgrade setuptools pip

  Found existing installation: setuptools 40.6.3
    Uninstalling setuptools-40.6.3:
      Successfully uninstalled setuptools-40.6.3
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\risha\\AppData\\Local\\Temp\\pip-uninstall-f88009si\\users\\risha\\anaconda3\\scripts\\pip.exe'
Consider using the `--user` option or check the permissions.



# Libraries

In [16]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.utils import shuffle
import sklearn.linear_model as Lm
from sklearn.linear_model import LinearRegression as Lr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
import requests
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from rasa_nlu.model import Metadata, Interpreter
from wit import Wit
import json

In [46]:
def getTextFromAudio(filename):
    client = speech.SpeechClient()

    file_name = os.path.join('Audio Files','Demo_4.flac')

    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code='en-US')

    response = client.recognize(config, audio)

    for result in response.results:
        return result.alternatives[0].transcript

In [47]:
getTextFromAudio("Demo_4.flac")

'hi I would like to order one burger and some fries'

In [44]:
client = Wit("QIPH5735KKLG3SRKLW6ULLPAA3FCYYIS")
resp = client.message('Can i order five Burger and a fries')
print(print(json.dumps(resp, indent=2)))


{
  "_text": "Can i order five Burger and a fries",
  "entities": {
    "item": [
      {
        "confidence": 0.7741024280796,
        "value": "fries",
        "type": "value"
      }
    ],
    "intent": [
      {
        "confidence": 0.98362402405517,
        "value": "Add"
      }
    ]
  },
  "msg_id": "1j88tGSXpxM23mKB3"
}
None


In [51]:
senten = []
senten.append('Can i order five Burger and a fries')
senten.append('I would like to order one large pizza and one soda')
senten.append('Please remove the burger from my order')
senten.append('Can you swap the burge for a cheese pizza')
senten.append('I Would like to get a coffee too')

In [55]:
WitClient = Wit("QIPH5735KKLG3SRKLW6ULLPAA3FCYYIS")
for i in senten:
    resp = WitClient.message(i)
    print(print(json.dumps(resp, indent=2)))


{
  "_text": "Can i order five Burger and a fries",
  "entities": {
    "item": [
      {
        "confidence": 1,
        "value": "burger",
        "type": "value"
      },
      {
        "confidence": 1,
        "value": "fries",
        "type": "value"
      }
    ],
    "intent": [
      {
        "confidence": 0.97615008349022,
        "value": "Add"
      }
    ]
  },
  "msg_id": "1ojGBkybiQ9k0784E"
}
None
{
  "_text": "I would like to order one large pizza and one soda",
  "entities": {
    "item": [
      {
        "confidence": 0.96937750911338,
        "value": "cheese pizza",
        "type": "value"
      }
    ],
    "intent": [
      {
        "confidence": 0.98781046493033,
        "value": "Add"
      }
    ]
  },
  "msg_id": "1IAWRHTX3jT4v7Jb5"
}
None
{
  "_text": "Please remove the burger from my order",
  "entities": {
    "item": [
      {
        "confidence": 1,
        "value": "burger",
        "type": "value"
      }
    ],
    "intent": [
      {
        "con

In [71]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

ImportError: cannot import name 'language' from 'google.cloud' (unknown location)

In [23]:
print('Credendtials from environ: {}'.format(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

Credendtials from environ: C:\Users\risha\Downloads\NLP - Food-96ceb070c2a2.json


In [5]:
!cat rasa_dataset.json

'cat' is not recognized as an internal or external command,
operable program or batch file.
